Step 1: Load The Data

First, I load the CSV files into Pandas DataFrame

In [1]:
import pandas as pd

# Load the CSV files
borough_recent = pd.read_csv('MPS Borough Level Crime (most recent 24 months).csv')
borough_historical = pd.read_csv('MPS Borough Level Crime (Historical).csv')

ward_recent = pd.read_csv('MPS Ward Level Crime (most recent 24 months).csv')
ward_historical = pd.read_csv('MPS Ward Level Crime (Historical).csv')

lsoa_recent = pd.read_csv('MPS LSOA Level Crime (most recent 24 months).csv')
lsoa_historical = pd.read_csv('MPS LSOA Level Crime (Historical).csv')

# Load your existing reported.csv file
reported_data = pd.read_csv('reported.csv')


C:\Users\jeffo\AppData\Local\Temp\ipykernel_18616\2611562565.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Step 2: Preprocess Data

I Clean and Preprocess the data.

In [2]:
import pandas as pd

# Load the CSV file
file_path = 'MPS Ward Level Crime (most recent 24 months).csv'
ward_recent = pd.read_csv(file_path)

# Display the first few rows to understand its structure
ward_recent.head()


,MajorText,MinorText,WardName,WardCode,LookUp_BoroughName,202202,202203,202204,202205,202206,...,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401
0,Arson and Criminal Damage,Arson,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,2,2,2,0,0,0,0,1,3
1,Arson and Criminal Damage,Criminal Damage,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,13,18,16,23,24,19,17,36,26
2,Burglary,Burglary Business and Community,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,1,4,2,2,4,6,3,1,4,2
3,Burglary,Domestic Burglary,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,7,5,8,14,14,1,10,11,12
4,Drug Offences,Drug Trafficking,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,5,3,2,5,3,3,0,4,3


I define a function to clean and preprocess the data.

In [4]:
def preprocess_ward_recent(df):
    # Drop rows with missing values
    df = df.dropna()
    
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # Clean text data in 'crime_type' column
    if 'crime_type' in df.columns:
        df['crime_type'] = df['crime_type'].str.lower().str.replace(r'[^\w\s]', '').str.strip()
    
    return df

# Apply preprocessing
ward_recent_cleaned = preprocess_ward_recent(ward_recent)

# Display the first few rows of the cleaned data
ward_recent_cleaned.head()

,majortext,minortext,wardname,wardcode,lookup_boroughname,202202,202203,202204,202205,202206,...,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401
0,Arson and Criminal Damage,Arson,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,2,2,2,0,0,0,0,1,3
1,Arson and Criminal Damage,Criminal Damage,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,13,18,16,23,24,19,17,36,26
2,Burglary,Burglary Business and Community,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,1,4,2,2,4,6,3,1,4,2
3,Burglary,Domestic Burglary,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,7,5,8,14,14,1,10,11,12
4,Drug Offences,Drug Trafficking,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,5,3,2,5,3,3,0,4,3


Convert Date Columns: Ensure any date columns are in the correct datetime format.

Feature Engineering: Create new features if needed (e.g., extract month/year from a date column).

In [5]:
def add_date_features(df):
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
    return df

# Apply feature engineering
ward_recent_cleaned = add_date_features(ward_recent_cleaned)

# Display the first few rows with the new date features
ward_recent_cleaned.head()

,majortext,minortext,wardname,wardcode,lookup_boroughname,202202,202203,202204,202205,202206,...,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401
0,Arson and Criminal Damage,Arson,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,2,2,2,0,0,0,0,1,3
1,Arson and Criminal Damage,Criminal Damage,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,13,18,16,23,24,19,17,36,26
2,Burglary,Burglary Business and Community,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,1,4,2,2,4,6,3,1,4,2
3,Burglary,Domestic Burglary,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,7,5,8,14,14,1,10,11,12
4,Drug Offences,Drug Trafficking,Heathrow Villages,E05013570,Aviation Security (SO18),0,0,0,0,0,...,0,5,3,2,5,3,3,0,4,3
